<a href="https://colab.research.google.com/github/chinmay002/Approach_any_ML/blob/main/XGBOOST_and_Target_Encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.model_selection import  StratifiedKFold
import xgboost as xgb
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

In [11]:
def run(fold):

  df = pd.read_csv('/content/cat_train_folds.csv')

  features = [col for col in df.columns if col not in ('id','kfold','target')]

  for col in features:
    df.loc[:,col] = df[col].astype(str).fillna('None')
  
  for col in features:
    lbl = LabelEncoder()
    lbl.fit(df[col])
    df[col] = lbl.transform(df[col])

  df_train = df[df['kfold']==fold]
  #print(df_train.head())
  df_test = df[df['kfold']!= fold]
#print(df_train[features].head())
  x_train = df_train[features].values
 # print(x_train)
  x_test = df_test[features].values

  model = xgb.XGBClassifier(max_depth = 7,n_estimators = 100)
  model.fit(x_train,df_train.target.values)
  pred = model.predict_proba(x_test)[:,1]

  print(f'AUC for fold {fold} : {metrics.roc_auc_score(df_test.target.values,pred)}')

for fold_ in range(3):
  run(fold_)  


AUC for fold 0 : 0.7440675306368237
AUC for fold 1 : 0.7427789398816267
AUC for fold 2 : 0.7431180829363566


In [12]:
df = pd.read_csv('/content/adult.csv')
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [13]:
df['income'].value_counts()

<=50K    24720
>50K      7841
Name: income, dtype: int64

In [15]:
df.select_dtypes(exclude='object').columns

Index(['age', 'fnlwgt', 'education.num', 'capital.gain', 'capital.loss',
       'hours.per.week'],
      dtype='object')

In [18]:
df['kfold'] = -1

df = df.sample(frac=1).reset_index(drop=True)
y = df.income.values
kf = StratifiedKFold(n_splits=5)

for index,(train,valid) in enumerate(kf.split(X=df,y=y)):
  df.loc[valid,'kfold'] = index

In [28]:
df.to_csv('sensus_folds.csv',index = False)

In [30]:
df = pd.read_csv('/content/sensus_folds.csv').iloc[:,1:]
df.head()
#df['kfold'].value_counts()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income,kfold
0,26,Private,244495,9th,5,Never-married,Other-service,Not-in-family,White,Male,0,0,35,United-States,<=50K,0
1,19,Private,64112,12th,8,Never-married,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,<=50K,0
2,37,Federal-gov,325538,Masters,14,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,?,>50K,0
3,40,Private,187376,Some-college,10,Married-civ-spouse,Sales,Husband,White,Male,0,0,60,United-States,>50K,0
4,41,Private,200652,9th,5,Divorced,Other-service,Other-relative,White,Female,0,0,35,United-States,<=50K,0


In [51]:
def run(fold):
  df = pd.read_csv('/content/sensus_folds.csv')

  num_col = ['age', 'fnlwgt', 'education.num', 'capital.gain', 'capital.loss','hours.per.week']

  df = df.drop(num_col,axis=1)     
  tar_map = {'<=50K':0,'>50K':1}

  df.loc[:,'income'] = df['income'].map(tar_map)

  features = [col for col in df.columns if col not in ('Unnamed: 0','kfold','income')]
  print(features)
  for col in features:
    df.loc[:,col] = df[col].astype(str).fillna('NONE')
  
  df_train = df[df['kfold']==fold].reset_index(drop=True)
  df_test = df[df['kfold']!=fold].reset_index(drop=True)
  #print(df_train.head())
  
  
  ohe = OneHotEncoder()
  full_data = pd.concat([df_train[features],df_test[features]],axis=0)
  #print('full_head')
  #print(full_data.head())
  
  ohe.fit(full_data[features])
 
  x_train = ohe.transform(df_train[features])
  x_test = ohe.transform(df_test[features])

  model = LogisticRegression()
  model.fit(x_train,df_train.income.values)

  pred = model.predict_proba(x_test)[:,1]

  print(f'AUC for fold {fold} : {metrics.roc_auc_score(df_test.income.values,pred)}')


  
for fold in range(5):
  run(5)


['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country']


ValueError: ignored

In [60]:
def run(fold):
  df = pd.read_csv('/content/sensus_folds.csv')
  num_cols = ["fnlwgt","age","capital.gain","capital.loss","hours.per.week" ]
  
  target_mapping = {"<=50K": 0,">50K": 1}
  df.loc[:, "income"] = df.income.map(target_mapping)
  
  features = [f for f in df.columns if f not in ("kfold", "income")]
  
  # fill all NaN values with NONE
  # note that I am converting all columns to "strings"
  # it doesnt matter because all are categories
  for col in features:
  # do not encode the numerical columns
    if col not in num_cols:
      df.loc[:, col] = df[col].astype(str).fillna("NONE")
    # now its time to label encode the features
  
  for col in features:
    if col not in num_cols:
  # initialize LabelEncoder for each feature column
      lbl = LabelEncoder()
  # fit label encoder on all data
      lbl.fit(df[col])
  # transform all the data
      df.loc[:, col] = lbl.transform(df[col])
  
  
  # get training data using folds
  df_train = df[df.kfold != fold].reset_index(drop=True)
  
  # get validation data using folds
  df_valid = df[df.kfold == fold].reset_index(drop=True)
  
  x_train = df_train[features].values
  # get validation data
  x_valid = df_valid[features].values
  # initialize xgboost model
  model = xgb.XGBClassifier(n_jobs=-1)
  # fit model on training data (ohe)
  model.fit(x_train, df_train.income.values)
  
  # predict on validation data
  # we need the probability values as we are calculating AUC
  # we will use the probability of 1s
  valid_preds = model.predict_proba(x_valid)[:, 1]
  # get roc auc score
  auc = metrics.roc_auc_score(df_valid.income.values, valid_preds)
  # print auc
  print(f"Fold = {fold}, AUC = {auc}")
for i in range(5):
  run(i)  

Fold = 0, AUC = 0.9225981728307974
Fold = 1, AUC = 0.9081831951530612
Fold = 2, AUC = 0.9232337769755962
Fold = 3, AUC = 0.9216302305206063
Fold = 4, AUC = 0.9326523388481608


In [60]:
def run(fold):
  df = pd.read_csv('/content/sensus_folds.csv')
  num_cols = ["fnlwgt","age","capital.gain","capital.loss","hours.per.week" ]
  
  target_mapping = {"<=50K": 0,">50K": 1}
  df.loc[:, "income"] = df.income.map(target_mapping)
  
  features = [f for f in df.columns if f not in ("kfold", "income")]
  
  # fill all NaN values with NONE
  # note that I am converting all columns to "strings"
  # it doesnt matter because all are categories
  for col in features:
  # do not encode the numerical columns
    if col not in num_cols:
      df.loc[:, col] = df[col].astype(str).fillna("NONE")
    # now its time to label encode the features
  
  for col in features:
    if col not in num_cols:
  # initialize LabelEncoder for each feature column
      lbl = LabelEncoder()
  # fit label encoder on all data
      lbl.fit(df[col])
  # transform all the data
      df.loc[:, col] = lbl.transform(df[col])
  
  
  # get training data using folds
  df_train = df[df.kfold != fold].reset_index(drop=True)
  
  # get validation data using folds
  df_valid = df[df.kfold == fold].reset_index(drop=True)
  
  x_train = df_train[features].values
  # get validation data
  x_valid = df_valid[features].values
  # initialize xgboost model
  model = xgb.XGBClassifier(n_jobs=-1)
  # fit model on training data (ohe)
  model.fit(x_train, df_train.income.values)
  
  # predict on validation data
  # we need the probability values as we are calculating AUC
  # we will use the probability of 1s
  valid_preds = model.predict_proba(x_valid)[:, 1]
  # get roc auc score
  auc = metrics.roc_auc_score(df_valid.income.values, valid_preds)
  # print auc
  print(f"Fold = {fold}, AUC = {auc}")
for i in range(5):
  run(i)  

In [64]:
list(itertools.combinations([1,2,3,4],2))

[(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]

In [69]:
#Feature Engineering
import itertools
import pandas as pd
import xgboost as xgb

def feature_engineering(df,cat_cols):
  '''
  Args : Takes a combination fo cat columns
  returns
  '''
  combi = list(itertools.combinations(cat_cols,2))
  for c1,c2 in combi:
    df.loc[:,c1+'_'+c2] = df[c1].astype(str)+ '_' +df[c2].astype(str)
  return df  

In [70]:
def run(fold):
  df = pd.read_csv('/content/sensus_folds.csv')
  num_cols = ["fnlwgt","age","capital.gain","capital.loss","hours.per.week" ]
  
  target_mapping = {"<=50K": 0,">50K": 1}
  df.loc[:, "income"] = df.income.map(target_mapping)
  
  features = [f for f in df.columns if f not in ("kfold", "income")]
  
  cat_cols = [col for col in features if col not in num_cols]
  
  df = feature_engineering(df,cat_cols)
  
  # fill all NaN values with NONE
  # note that I am converting all columns to "strings"
  # it doesnt matter because all are categories
  for col in features:
  # do not encode the numerical columns
    if col not in num_cols:
      df.loc[:, col] = df[col].astype(str).fillna("NONE")
    # now its time to label encode the features
  


  for col in features:
    if col not in num_cols:
  # initialize LabelEncoder for each feature column
      lbl = LabelEncoder()
  # fit label encoder on all data
      lbl.fit(df[col])
  # transform all the data
      df.loc[:, col] = lbl.transform(df[col])
  
  
  # get training data using folds
  df_train = df[df.kfold != fold].reset_index(drop=True)
  
  # get validation data using folds
  df_valid = df[df.kfold == fold].reset_index(drop=True)
  
  x_train = df_train[features].values
  # get validation data
  x_valid = df_valid[features].values
  # initialize xgboost model
  model = xgb.XGBClassifier(n_jobs=-1)
  # fit model on training data (ohe)
  model.fit(x_train, df_train.income.values)
  
  # predict on validation data
  # we need the probability values as we are calculating AUC
  # we will use the probability of 1s
  valid_preds = model.predict_proba(x_valid)[:, 1]
  # get roc auc score
  auc = metrics.roc_auc_score(df_valid.income.values, valid_preds)
  # print auc
  print(f"Fold = {fold}, AUC = {auc}")
for i in range(5):
  run(i)  

Fold = 0, AUC = 0.9225981728307974
Fold = 1, AUC = 0.9081831951530612
Fold = 2, AUC = 0.9232337769755962
Fold = 3, AUC = 0.9216302305206063
Fold = 4, AUC = 0.9326523388481608


**Target Encoding**
 It means that the first thing you do is create the folds, and then use those
folds to create target encoding features for different columns of the data in the same
way you fit and predict the model on folds. So, if you have created 5 folds, you have
to create target encoding 5 times such that in the end, you have encoding for
variables in each fold which are not derived from the same fold. And then when
you fit your model, you must use the same folds again. Target encoding for unseen
test data can be derived from the full training data or can be an average of all the 5
folds.

In [89]:
import copy

def mean_target_encoding(data):
  
  df = copy.deepcopy(data)
  df = df.iloc[:,1:]

  num_cols = ["fnlwgt","age","capital.gain","capital.loss","hours.per.week" ]
  
  target_mapping = {"<=50K": 0,">50K": 1}
  
  df.loc[:, "income"] = df.income.map(target_mapping)
  
  features = [f for f in df.columns if f not in ('Unnamed: 0',"kfold", "income")]
  print(features)
  
  cat_cols = [col for col in features if col not in num_cols]
  
  for col in features:
    if col not in num_cols:
      df.loc[:, col] = df[col].astype(str).fillna("NONE")
  
  for col in features:
    if col not in num_cols:
      lbl = LabelEncoder()
      lbl.fit(df[col])
      df.loc[:, col] = lbl.transform(df[col])
  
  # a list to store 5 validation dataframes:
  encoded_dfs = []

  #go over all folds
  print(df.head())
  for fold in range(5):

    #fetch trainng and validation data
    df_train = df[df['kfold']!=fold].reset_index(drop=True)
    df_test = df[df['kfold']==fold].reset_index(drop=True)

    for col in cat_cols:
      mapping_dic = dict(df_train.groupby(col)['income'].mean())
      df_test.loc[:,col+'_enc'] = df_test[col].map(mapping_dic)
    encoded_dfs.append(df_test)  
  encoded_df = pd.concat(encoded_dfs,axis=0)
  return encoded_df


df = pd.read_csv("/content/sensus_folds.csv")
# create mean target encoded categories and
# munge data
df_1 = mean_target_encoding(df)




  

['age', 'workclass', 'fnlwgt', 'education', 'education.num', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'capital.gain', 'capital.loss', 'hours.per.week', 'native.country']
   age  workclass  fnlwgt  education  education.num  marital.status  \
0   26          4  244495          6             11               4   
1   19          4   64112          2             14               4   
2   37          1  325538         12              5               2   
3   40          4  187376         15              1               2   
4   41          4  200652          6             11               0   

   occupation  relationship  race  sex  capital.gain  capital.loss  \
0           8             1     4    1             0             0   
1           1             1     4    0             0             0   
2          10             0     4    1             0             0   
3          12             0     4    1             0             0   
4           8             2    

In [105]:
def run(df,fold):
  df = pd.read_csv('/content/sensus_folds.csv')
  num_cols = ["fnlwgt","age","capital.gain","capital.loss","hours.per.week" ]
  
  target_mapping = {"<=50K": 0,">50K": 1}
  df.loc[:, "income"] = df.income.map(target_mapping)
  
  features = [f for f in df.columns if f not in ("kfold", "income")]
  
  cat_cols = [col for col in features if col not in num_cols]
  
  df = feature_engineering(df,cat_cols)
  
  # fill all NaN values with NONE
  # note that I am converting all columns to "strings"
  # it doesnt matter because all are categories
  for col in features:
  # do not encode the numerical columns
    if col not in num_cols:
      df.loc[:, col] = df[col].astype(str).fillna("NONE")
    # now its time to label encode the features
  


  for col in features:
    if col not in num_cols:
  # initialize LabelEncoder for each feature column
      lbl = LabelEncoder()
  # fit label encoder on all data
      lbl.fit(df[col])
  # transform all the data
      df.loc[:, col] = lbl.transform(df[col])
  
  
  # get training data using folds
  df_train = df[df.kfold != fold].reset_index(drop=True)
  
  # get validation data using folds
  df_valid = df[df.kfold == fold].reset_index(drop=True)
  
  x_train = df_train[features].values
  # get validation data
  x_valid = df_valid[features].values
  # initialize xgboost model
  model = xgb.XGBClassifier(n_jobs=-1)
  # fit model on training data (ohe)
  model.fit(x_train, df_train.income.values)
  
  # predict on validation data
  # we need the probability values as we are calculating AUC
  # we will use the probability of 1s
  valid_preds = model.predict_proba(x_valid)[:, 1]
  # get roc auc score
  auc = metrics.roc_auc_score(df_valid.income.values, valid_preds)
  # print auc
  print(f"Fold = {fold}, AUC = {auc}")

df= pd.read_csv('sensus_folds.csv')
df = mean_target_encoding(df)
for i in range(5):
  run(df,i)  

['age', 'workclass', 'fnlwgt', 'education', 'education.num', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'capital.gain', 'capital.loss', 'hours.per.week', 'native.country']
   age  workclass  fnlwgt  education  education.num  marital.status  \
0   26          4  244495          6             11               4   
1   19          4   64112          2             14               4   
2   37          1  325538         12              5               2   
3   40          4  187376         15              1               2   
4   41          4  200652          6             11               0   

   occupation  relationship  race  sex  capital.gain  capital.loss  \
0           8             1     4    1             0             0   
1           1             1     4    0             0             0   
2          10             0     4    1             0             0   
3          12             0     4    1             0             0   
4           8             2    